# API Calls and raw labels

## The Weather API 

In [3]:
import pandas as pd
import requests as r

In [4]:
# The API Endpoint for Historical Weather
url: str = "https://archive-api.open-meteo.com/v1/archive"

# Define parameters (Western Hub Proxy: Harrisburg, PA)
params: dict = {
    "latitude": 40.27,
    "longitude": -76.88,
    "start_date": "2024-01-01",
    "end_date": "2024-12-31",
    "hourly": "temperature_2m,wind_speed_10m,shortwave_radiation", # Radiation = Sunlight
    "timezone": "America/New_York" # Critical: Match PJM EPT Timezone
}

# Fetch the data
response = r.get(url, params=params)

# Process the JSON Response
if response.status_code == 200: # could used response.raise_for_status() without if-else
    data = response.json()

    # Extract hourly data
    hourly: dict = data["hourly"]
    
    # Create DataFrame
    df_weather: pd.DataFrame = pd.DataFrame({
        "time_ept": hourly["time"],
        "temp_c": hourly["temperature_2m"],
        "wind_kph": hourly["wind_speed_10m"],
        "solar_radiation": hourly["shortwave_radiation"]
    })

    print(f"Success! Downloaded {len(df_weather)} rows of weather data.")
    display(df_weather.head())
else:
    print("Error:", response.status_code, response.text)

Success! Downloaded 8784 rows of weather data.


,time_ept,temp_c,wind_kph,solar_radiation
0,2024-01-01T00:00,3.0,5.0,0.0
1,2024-01-01T01:00,3.0,5.0,0.0
2,2024-01-01T02:00,3.0,5.2,0.0
3,2024-01-01T03:00,2.7,4.1,0.0
4,2024-01-01T04:00,2.2,3.5,0.0


### Step 1: Connect to the Database

##### Debugging 'localhost' issues

In [5]:
import os
from dotenv import load_dotenv

# Force reload the .env file
load_dotenv(override=True)

host = os.getenv("DB_HOST")
print(f"Raw Host value: {repr(host)}") 
# If it prints ' localhost' or '\nlocalhost', we found the bug.

Raw Host value: 'localhost'


In [12]:
import psycopg2
from dotenv import load_dotenv

load_dotenv()


conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    port=os.getenv("DB_PORT")
)
print("Connected!")



Connected!


In [7]:
# Database connection setup
from sqlalchemy import create_engine
import sys
import os

# 1. Add the project root to the path so we can import 'src'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# 2. Create teh Database Engine
from src.config import DB_CONNECTION_STRING

print("Connecting to Database...")

# 3. Read the SQL View created earlier
query: str = "SELECT * FROM pjm_market.features_v"
df_prices: pd.DataFrame = pd.read_sql(query, con=DB_CONNECTION_STRING)

print(f"Loaded {len(df_prices)} rows from SQL.")
display(df_prices.head())

Connecting to Database...
Loaded 8785 rows from SQL.


,datetime_beginning_ept,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h
0,2024-01-01 00:00:00,31.3827,0.0,1.0,1.0,NaN,NaN,NaN
1,2024-01-01 01:00:00,20.0838,1.0,1.0,1.0,31.3827,NaN,31.38270
2,2024-01-01 02:00:00,17.6052,2.0,1.0,1.0,20.0838,NaN,25.73325
3,2024-01-01 03:00:00,19.7673,3.0,1.0,1.0,17.6052,NaN,23.02390
4,2024-01-01 04:00:00,17.0687,4.0,1.0,1.0,19.7673,NaN,22.20975


### Step 2: Clean and Merge DataFrames

In [8]:
# Convert Weather timestamps from string to datetime
df_weather["time_ept"] = pd.to_datetime(df_weather["time_ept"])

# Ensure SQL timestamps are datetimes (usually they are, but let's be safe)
df_prices["datetime_beginning_ept"] = pd.to_datetime(df_prices["datetime_beginning_ept"])

# Merge (Inner Join)
df_merged = df_prices.merge(
    df_weather,
    left_on="datetime_beginning_ept",
    right_on="time_ept",
    how="inner"
)

# Drop duplicate time column
df_merged = df_merged.drop("time_ept", axis=1) # columns=["time_ept"]

print(f"Final Dataset Shape: {df_merged.shape}")
display(df_merged.head())

Final Dataset Shape: (8785, 11)


,datetime_beginning_ept,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
0,2024-01-01 00:00:00,31.3827,0.0,1.0,1.0,NaN,NaN,NaN,3.0,5.0,0.0
1,2024-01-01 01:00:00,20.0838,1.0,1.0,1.0,31.3827,NaN,31.38270,3.0,5.0,0.0
2,2024-01-01 02:00:00,17.6052,2.0,1.0,1.0,20.0838,NaN,25.73325,3.0,5.2,0.0
3,2024-01-01 03:00:00,19.7673,3.0,1.0,1.0,17.6052,NaN,23.02390,2.7,4.1,0.0
4,2024-01-01 04:00:00,17.0687,4.0,1.0,1.0,19.7673,NaN,22.20975,2.2,3.5,0.0


In [9]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetime_beginning_ept  8785 non-null   datetime64[ns]
 1   price_actual            8785 non-null   float64       
 2   hour_of_day             8785 non-null   float64       
 3   day_of_week             8785 non-null   float64       
 4   month                   8785 non-null   float64       
 5   price_1h_ago            8784 non-null   float64       
 6   price_24h_ago           8761 non-null   float64       
 7   avg_price_last_24h      8784 non-null   float64       
 8   temp_c                  8785 non-null   float64       
 9   wind_kph                8785 non-null   float64       
 10  solar_radiation         8785 non-null   float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 755.1 KB


### Step 3: Save My Work

In [10]:
# Save to the processed folder
output_path = "../data/processed/merged_data.csv"
df_merged.to_csv(output_path, index=False)

print(f"Data successfully saved to {output_path}")

Data successfully saved to ../data/processed/merged_data.csv
